# Model selection

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [3]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [4]:
import seaborn as sns

Import data from csv files into a dataframe

In [5]:
data = pd.read_csv('Combined_all_3rd_fit.csv', encoding='cp1252')
print(data.columns.values)

['Name' 'Coef_a' 'Coef_b' 'Coef_c' 'Coef_d' 'A_site' 'B_site' 'X_site'
 'Spacegroup' 'Ehull' 'BulkModulus' 'Energy' 'ZPE' 's_A' 's_B' 's_X'
 'density' 'mean_A2B' 'mean_A2X' 'mean_B2X' 'mean_X2X' 'std_A2B' 'std_A2X'
 'std_B2X' 'std_X2X' 'E_coh' 'TF' 'OF' 'A_Z' 'B_Z' 'X_Z' 'A_M' 'B_M' 'X_M'
 'A_G' 'B_G' 'X_G' 'A_IEI' 'B_IEI' 'X_IEI' 'A_IEII' 'B_IEII' 'X_IEII'
 'A_EA' 'B_EA' 'X_EA' 'A_ChiP' 'B_ChiP' 'X_ChiP' 'A_ChiA' 'X_ChiA'
 'A_Rvdw' 'B_Rvdw' 'X_Rvdw' 'A_Rc' 'B_Rc' 'X_Rc' 'A_Ra' 'B_Ra' 'X_Ra'
 'A_MP' 'B_MP' 'X_MP' 'A_BP' 'B_BP' 'X_BP' 'A_Rho' 'B_Rho' 'A_MV' 'B_MV'
 'X_MV' 'A_Hf' 'B_Hf' 'X_Hf' 'A_Hv' 'B_Hv' 'X_Hv' 'A_Kappa' 'B_Kappa'
 'X_Kappa' 'A_CvM' 'B_CvM' 'X_CvM' 'A_B' 'B_B' 'X_B' 'A_MendeleevNo'
 'B_MendeleevNo' 'X_MendeleevNo']


Drop the unnecessary columns and fill the empty cells with zero

In [6]:
data.drop(['Name', 'A_site', 'B_site', 'X_site', 'Spacegroup','BulkModulus',
           'Ehull','Energy','ZPE','Coef_b', 'Coef_c', 'Coef_a'], axis=1, inplace = True)
data.fillna(0, inplace= True)
d=data.copy()
columns = list(d.columns.values)
print(d.shape)

(80, 77)


Set the target variable to machine learn

In [7]:
target = 'Coef_d'

Define various methods to be used for buildig and validating the models

In [8]:
"""
    Functions to perform scaling
    
    """
def standard_scaling(target):
    data_std=d.copy()
    data_std[columns]= StandardScaler().fit_transform(d[columns])
    Y = data_std[target] 
    X = data_std.drop([target], axis=1)
    return X,Y

def minmax_scaling(target):
    data_mm=d.copy()
    data_mm[columns]= MinMaxScaler().fit_transform(d[columns])
    Y = data_mm[target] 
    X = data_mm.drop([target], axis=1)
    return X,Y

In [9]:
"""
    Function to build machine learning models by hyper parameter tuning. It also plots scatter plot
    
    """
def build_cv_model(X,Y,b_drop=False,target='Coef_d'):
    if b_drop :
        correlated_features = ["A_Rc","A_Ra","A_M","A_MP","A_MV","A_MendeleevNo","A_Hf","A_Hv","B_Rc",
                       "B_Rvdw","B_M","B_BP","B_MendeleevNo","B_Hv","X_Rc","X_Rvdw","X_M","X_BP","X_MP",
                       "X_MendeleevNo","X_Hf","X_Hv","X_G","X_B","X_CvM","X_ChiP"]
        X.drop(labels=correlated_features, axis=1, inplace=True)
        
    print("The shape of X is ",X.shape)
    
    ml_model, cv_results = train_model(X,Y,hyperparams=hyperparams, cv=True, return_cv=True)
    
    return ml_model

In [10]:
def run_model(X,Y,model,target='Coef_d'):
    rmse,r2 = run_cv(model, X, Y, n_cv = 5)
    print("-----------------------------------------------------")
    print("The R2 is ",r2)
    print("The RMSE is ",rmse)

In [11]:
"""
    Function to perform feature elimination using select K best of RFE method. It also prints the 
    feature scores obtained as:
    SelectKBest: The scores obtained from the selector
    RFE: The feature importance obtained from the RFR model 
    
    """
def feature_elimination(X,Y,n,estimator,method='skb'):
    if method=='skb':
        bestfeatures = SelectKBest(score_func=f_regression, k=10)
        fit = bestfeatures.fit(X,Y)
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(X.columns)
        #concat two dataframes for better visualization 
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Feature','Score']  #naming the dataframe columns
        X=X[featureScores.nlargest(n,'Score')['Feature'].values]
    elif method=='rfe':
        selector = RFE(estimator, n_features_to_select=n, step=1)
        selector = selector.fit(X, Y)
        dfscores = pd.DataFrame(selector.ranking_)
        dfcolumns = pd.DataFrame(selector.feature_names_in_)
        #concat two dataframes for better visualization 
        featureSelection = pd.concat([dfcolumns,dfscores],axis=1)
        featureSelection.columns = ['Feature','Score']  #naming the dataframe columns
        X=X[featureSelection.nsmallest(n,'Score')['Feature'].values]
        estimator.fit(X,Y)
        importance = pd.DataFrame(estimator.feature_importances_)
        featureNames = pd.DataFrame(X.columns.values)
        featureScores = pd.concat([featureNames,importance],axis=1)
        featureScores.columns = ['Feature','Score']  #naming the dataframe columns
        
    print(featureScores)
    print("-----------------------------------------------------")
    return X

In [12]:
def train_model(X, y, hyperparams = None, cv = False, return_cv = False):
    """
    Function to train the ML model on the given data X (features) and y (target property). 
    If hyperparams argument is passed, all possible combinations of alpha and kernel values
    will be tried to find combination with minimum CV error. The final  model is trained
    using the fixed kernel and alpha params as determined using CV.
    
    """
    
    if cv:
        cv_results = {'kernel':[],'degree':[],'C':[],
                      'gamma':[],'epsilon':[],'cv_rmse':[],'cv_r2':[]}
        for k,d,c,g,e in itertools.product(hyperparams['kernel'], 
                                           hyperparams['degree'],hyperparams['C'],
                                          hyperparams['gamma'],hyperparams['epsilon']):

            svr = SVR(kernel=k, degree=d, C=c, gamma=g, epsilon=e)

            cv_error, cv_r2 = run_cv(svr, X, y, n_cv = 5)
            cv_results['cv_rmse'].append(cv_error)
            cv_results['cv_r2'].append(cv_r2)
            cv_results['kernel'].append(k)
            cv_results['degree'].append(d)
            cv_results['C'].append(c)
            cv_results['gamma'].append(g)
            cv_results['epsilon'].append(e)

        cv_results = pd.DataFrame(cv_results)
        cv_results = cv_results.sort_values('cv_rmse')
        k_opt = cv_results.iloc[0]['kernel']
        d_opt = cv_results.iloc[0]['degree']
        c_opt = cv_results.iloc[0]['C']
        g_opt = cv_results.iloc[0]['gamma']
        e_opt = cv_results.iloc[0]['epsilon']
        
    else:
        k_opt = hyperparams['kernel'][0]
        d_opt = hyperparams['degree'][0]
        c_opt = hyperparams['C'][0]
        g_opt = hyperparams['gamma'][0]
        e_opt = hyperparams['epsilon'][0]

    svr = SVR(kernel=k_opt, degree=d_opt, C=c_opt, gamma=g_opt, epsilon=e_opt)
    model = svr.fit(X,y)
    print("-----------------------------------------------------")
    print("The R2 for ",k_opt,",",d_opt,",",c_opt,",",g_opt,",",e_opt," is ",cv_results.iloc[0]['cv_r2'])
    print("The RMSE for ",k_opt,",",d_opt,",",c_opt,",",g_opt,",",e_opt," is ",cv_results.iloc[0]['cv_rmse'])
    return [model, cv_results] if return_cv else model


def run_cv(ml_model, X, y, n_cv = 5):
    """
    Function to run Cross-validation
    """
    kf = KFold(n_splits=n_cv,shuffle=True,random_state=50)
    y_val = []
    y_pred = []

    for idx, (train, val) in enumerate(kf.split(X)):
        
        X_cv_train = X.values[train]
        X_cv_val = X.values[val]

        y_cv_train = y.values[train]
        y_cv_val = y.values[val]    

        # Model fit and prediction
        model = ml_model.fit(X_cv_train, y_cv_train)
        y_pred_val = model.predict(X_cv_val)
        
        y_val.append(y_cv_val)
        y_pred.append(y_pred_val)

    # Computing errors
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = r2_score(y_val, y_pred)
    
    return rmse, r2

Define the hyperparameters to be tuned for the given algorithm

In [13]:
hyperparams={'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'degree':[2,3],
            'C':[0.1,0.5,1,2,4],
            'gamma':['auto','scale'],
            'epsilon':[0.01,0.1,0.5,1,2,4]}

## Without scaling

In [14]:
data_std=d.copy()
Y = data_std[target] 
X = data_std.drop([target], axis=1)

In [ ]:
ml_model=build_cv_model(X,Y,False,target)

The shape of X is  (80, 76)


### Drop correlated features

In [ ]:
ml_model_prime=build_cv_model(X,Y,True,target)

### Feature elimination

In [ ]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

## Standard scaling

In [ ]:
X,Y = standard_scaling(target)
ml_model=build_cv_model(X,Y,False,target)

### Drop correlated features

In [ ]:
ml_model_prime=build_cv_model(X,Y,True,target)

### Feature elimination

In [ ]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

## Minmax scaling

In [ ]:
X,Y = minmax_scaling(target)
ml_model=build_cv_model(X,Y,False,target)

### Drop correlated features

In [ ]:
ml_model_prime=build_cv_model(X,Y,True,target)

### Feature elimination

In [ ]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

In [ ]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)